### Executive Summary
Dear CEO,
Thank you for the oppurtunity to work with you on this special project to better understand the movie industry. We have conducted a series of analysis to better understand top selling movies, fast growing georaphies and who to partner with in order for Microsoft to enter into the movie industry.

This analysis provides you with insights by answering the following questions:
    1. What is the movie industry trend like over the past 2 decades?
    2. What is the correlation production budget and worldwide gross?
    3. Which genres have had the highest grossing revenue and ROI?
    4. Which genres have had the highest ratings?
    5. Is there any correlation popularity and revenue?
    6. Who are the best directors to partner with in winning genres?
    7. What are the common runtimes by genre?
    8. Which studio should Microsoft partner with?

### Importing Python libraries

The sets of codes below have been carefully written to import a number of python libraries needed to import, interprete and manipulate data in order to generate inisights needed.

In [1]:
#importing matplot
import matplotlib.pyplot as plt 

# Set plot space as inline for inline plots display
%matplotlib inline

#importing numpy
import numpy as np 

# importing pandas into jupyter notebook
import pandas as pd

### Importing data from SQL

Running the below codes creates a connection between python and the SQL database and enables python to pull data from the "Movies" database. 
Three tables (*Box Office Mojo*; *IMDB*; *Rotten Tomatoes*) were merged in SQL into a larger database and the inported into pandaas dataframe. This enabled the team generate depper insights across databases.

In [6]:
#importing SQL connector
import mysql.connector

con = mysql.connector.connect(user='hbs-student', password='super-secure-password?',
                              host='hbs-ds-060120.c1doesqrid0e.us-east-1.rds.amazonaws.com',
                              database='movies')

#importing merged database (*Box Office Mojo*; *IMDB*; *Rotten Tomatoes*)
df = pd.read_sql_query("""
                         SELECT  title, studio, bom_movie_gross.domestic_gross, 
                                 foreign_gross, year, tconst, runtime_minutes, genres, 
                                 averagerating, numvotes, directors, writers, production_budget
                         FROM movies.bom_movie_gross
                         JOIN imdb_title_basics
                         ON title = primary_title
                         JOIN imdb_title_ratings
                         USING (tconst)
                         JOIN imdb_title_crew
                         USING (tconst)
                         JOIN tn_movie_budgets
                         ON title = movie;
                         """, con)
# getting first 5 rows of imported data
df.head()

,title,studio,domestic_gross,foreign_gross,year,tconst,runtime_minutes,genres,averagerating,numvotes,directors,writers,production_budget
0,Toy Story 3,BV,415000000.0,652000000.0,2010,tt0435761,103.0,"Adventure,Animation,Comedy",8.3,682218,nm0881279,"nm0005124,nm0004056,nm0881279,nm1578335","$200,000,000"
1,Inception,WB,292600000.0,535700000.0,2010,tt1375666,148.0,"Action,Adventure,Sci-Fi",8.8,1841066,nm0634240,nm0634240,"$160,000,000"
2,Shrek Forever After,P/DW,238700000.0,513900000.0,2010,tt0892791,93.0,"Adventure,Animation,Comedy",6.3,167532,nm0593610,"nm0825308,nm0458441,nm0501359,nm0011470,nm0254...","$165,000,000"
3,The Twilight Saga: Eclipse,Sum.,300500000.0,398000000.0,2010,tt1325004,124.0,"Adventure,Drama,Fantasy",5.0,211733,nm1720541,"nm0742279,nm2769412","$68,000,000"
4,Iron Man 2,Par.,312400000.0,311500000.0,2010,tt1228705,124.0,"Action,Adventure,Sci-Fi",7.0,657690,nm0269463,"nm0857620,nm0498278,nm1411347,nm1293367,nm0456158","$170,000,000"


### Cleaning the dataset

The following methodology was leveraged in order to clean the dataset into a workable version:
* Tranformed old 'Directors' column from (one column containing multiple values) to multiple columns contianing one value)
* Created new total_gross column (domestic_gross + foreign_gross)
* Created new ROI column  (total_gross / production_budget -1)
* Removed rows containing empty/ no values for total_gross
* Converted columns (production_budget; total_gross; domestic_gross; foreign_gross) into integers

In [7]:
# creating new database with split value columns
directors = df["directors"].str.split(",", n = 1, expand = True)
# making separate director columns from new data frame 
df["director1"]= directors[0] 
df["director2"]= directors[1]
# Dropping old directors columns 
df.drop(columns =["directors"], inplace = True)

# creating total gross column
df["total_gross"]= df['domestic_gross'] + df['foreign_gross']

# removing empty cells
df['total_gross'].replace('', np.nan, inplace=True)
df.dropna(subset=['total_gross'], inplace=True)

# cleaning production budget into integers
df['production_budget'] = df['production_budget'].str.replace(',', '')
df['production_budget'] = df['production_budget'].str.replace('$', '')
df['production_budget'] = df['production_budget'].astype(int)

# creating ROI column
df["ROI"]= (df['total_gross'] / df['production_budget']) -1 

# converting columns to integer
df['total_gross'] = df['total_gross'].astype(int)
df['domestic_gross'] = df['domestic_gross'].astype(int)
df['foreign_gross'] = df['foreign_gross'].astype(int)
df.head()

,title,studio,domestic_gross,foreign_gross,year,tconst,runtime_minutes,genres,averagerating,numvotes,writers,production_budget,director1,director2,total_gross,ROI
0,Toy Story 3,BV,415000000,652000000,2010,tt0435761,103.0,"Adventure,Animation,Comedy",8.3,682218,"nm0005124,nm0004056,nm0881279,nm1578335",200000000,nm0881279,None,1067000000,4.335000
1,Inception,WB,292600000,535700000,2010,tt1375666,148.0,"Action,Adventure,Sci-Fi",8.8,1841066,nm0634240,160000000,nm0634240,None,828300000,4.176875
2,Shrek Forever After,P/DW,238700000,513900000,2010,tt0892791,93.0,"Adventure,Animation,Comedy",6.3,167532,"nm0825308,nm0458441,nm0501359,nm0011470,nm0254...",165000000,nm0593610,None,752600000,3.561212
3,The Twilight Saga: Eclipse,Sum.,300500000,398000000,2010,tt1325004,124.0,"Adventure,Drama,Fantasy",5.0,211733,"nm0742279,nm2769412",68000000,nm1720541,None,698500000,9.272059
4,Iron Man 2,Par.,312400000,311500000,2010,tt1228705,124.0,"Action,Adventure,Sci-Fi",7.0,657690,"nm0857620,nm0498278,nm1411347,nm1293367,nm0456158",170000000,nm0269463,None,623900000,2.670000


### Analysis

Below are results to analysis conducted. 

#### Question 1. What is the highest domestic grossing studios from 2010 – 2012?

In [6]:
# insert code for analysis and graph here

#### Question 2. What is the movie industry trend like over the past 2 decades?

#### Question 3. What is the correlation production budget and worldwide gross?

#### Question 4. Which genres have had the highest grossing revenue and ROI?

### Insights:
* njksh